In [ ]:
import os
import tarfile
from shutil import copyfile


# Fonction pour créer un répertoire s'il n'existe pas déjà
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

# Fonction pour séparer les images de CUB200 en jeu d'entraînement et de test
def separate_train_test(dataset_path, train_path, test_path):
    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        if classname.startswith('.'):
            continue
        # Créer des répertoires pour l'ensemble d'entraînement et de test
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            if file.startswith('.'):
                continue
            file_path = os.path.join(dataset_path, classname, file)
            if i < 15:
                # Copier les premiers 15 fichiers dans le répertoire de test
                copyfile(file_path, os.path.join(test_path, classname, file))
            else:
                # Copier les autres fichiers dans le répertoire d'entraînement
                copyfile(file_path, os.path.join(train_path, classname, file))
            i += 1

        class_index += 1

# Chemin du fichier tarball (TGZ) à extraire
tgz_file_path = '/content/CUB_200_2011.tar'

# Chemin du répertoire où les fichiers seront extraits
extraction_directory = '/content/CUB_200_2011/'

# Chemins pour les ensembles d'entraînement et de test
train_path = '/content/train/'
test_path = '/content/test/'

# Créez le répertoire d'extraction s'il n'existe pas
make_dir(extraction_directory)

!tar -xf /content/CUB_200_2011.tar -C /content/

# Chemin vers le dossier images dans le répertoire extrait
images_path = os.path.join(extraction_directory, 'images/')

# Séparez les données en ensembles d'entraînement et de test
separate_train_test(images_path, train_path, test_path)
print(f'Les données ont été séparées en ensembles d\'entraînement dans : {train_path} et de test dans : {test_path}')


Les données ont été séparées en ensembles d'entraînement dans : /content/train/ et de test dans : /content/test/


In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
import time






# Définir la transformation des données pour l'entraînement et le test
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [ ]:


# Load training data using ImageFolder with custom filter
train_data = ImageFolder(root=train_path, transform=transform_train)

# Load testing data using ImageFolder with custom filter
test_data = ImageFolder(root=test_path, transform=transform_test)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


Transformation avec Normalisation à partir des données

In [ ]:
# Fonction pour calculer la moyenne et l'écart type d'un dataset
def calculate_mean_std(loader):
    mean = torch.zeros(3)
    std = torch.zeros(3)
    num_samples = 0

    # Calculer les sommes des valeurs de pixels et des carrés des valeurs de pixels pour chaque canal
    for images, _ in loader:
        mean += torch.sum(images, dim=[0, 2, 3])
        std += torch.sum(images**2, dim=[0, 2, 3])
        num_samples += images.size(0)

    # Diviser par le nombre total d'échantillons et de pixels par canal
    mean /= (num_samples * images.size(2) * images.size(3))
    std = torch.sqrt(std / (num_samples * images.size(2) * images.size(3)) - mean**2)

    return mean, std

# Calculer la moyenne et l'écart type pour l'ensemble d'entraînement
train_mean, train_std = calculate_mean_std(train_loader)
print(f'Mean (train): {train_mean}')
print(f'Std (train): {train_std}')



# Vous pouvez maintenant utiliser ces valeurs dans vos transformations pour normaliser les données
transform_train_norm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=train_mean, std=train_std)
])

transform_test_norm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=train_mean, std=train_std)
])
# Load the training and test data
train_data_norm = ImageFolder(train_path, transform=transform_train_norm)
test_data_norm = ImageFolder(test_path, transform=transform_test_norm)

# Define the data loaders for training and testing
train_loader_norm = DataLoader(train_data_norm, batch_size=64, shuffle=True)
test_loader_norm = DataLoader(test_data_norm, batch_size=64, shuffle=False)

Mean (train): tensor([0.0110, 0.2021, 0.1252])
Std (train): tensor([0.9894, 0.9892, 1.1602])


Initialisation aléatoire par défaut:première normalisation

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
import time
model = models.resnet18(pretrained=False)
num_classes = len(train_data_norm.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_norm:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/5 - Loss: 5.1437
Epoch 2/5 - Loss: 4.7047
Epoch 3/5 - Loss: 4.3489
Epoch 4/5 - Loss: 4.0344
Epoch 5/5 - Loss: 3.7745


In [ ]:
def calculate_accuracy(loader, model):
    model.eval()
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for inputs, labels in loader:
            try:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)
            except OSError:
                # Ignore truncated images and continue
                continue
    accuracy = correct_predictions / total_samples * 100
    return accuracy
train_accuracy = calculate_accuracy(train_loader_norm, model)
print(train_accuracy)

15.395994538006372


initialisation aléatoire par défaut : deuxième Normalisation

In [ ]:
model = models.resnet18(pretrained=False)
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
num_epochs = 5  # Set the number of epochs you want to train
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")
train_accuracy= calculate_accuracy(train_loader, model)
print(train_accuracy)

Epoch 1/5 - Loss: 5.1531
Epoch 2/5 - Loss: 4.7466
Epoch 3/5 - Loss: 4.3243
Epoch 4/5 - Loss: 4.0076
Epoch 5/5 - Loss: 3.7081
12.152935821574875


le modèle pré-entraîné, mais en gelant («freeze») tous les paramètres de convolution(La deuxieme methode de normalisation)

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
import time







# Define the data transformation for training and testing
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the training and test data
train_data = ImageFolder(train_path, transform=transform_train)
test_data = ImageFolder(test_path, transform=transform_test)

# Define the data loaders for training and testing
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(pretrained=True)

# Freeze all convolutional parameters except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True  # Unfreeze the final fully connected layer

# Replace the final fully connected layer for the number of classes in the dataset
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
start_time = time.time()
for epoch in range(5):  # 5 epochs for demonstration
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{5} - Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")
train_accuracy = calculate_accuracy(train_loader, model)
print(train_accuracy)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5 - Loss: 4.3258
Epoch 2/5 - Loss: 2.6652
Epoch 3/5 - Loss: 1.9893
Epoch 4/5 - Loss: 1.6200
Epoch 5/5 - Loss: 1.3840
Training Time: 282.187495470047 seconds
75.59171597633136


le modèle pré-entraîné, mais en gelant («freeze») tous les paramètres de convolution (normalisation à partir des données)

In [ ]:

# Load the training and test data
train_data_norm = ImageFolder(train_path, transform=transform_train_norm)
test_data_norm = ImageFolder(test_path, transform=transform_test_norm)

# Define the data loaders for training and testing
train_loader_norm = DataLoader(train_data_norm, batch_size=64, shuffle=True)
test_loader_norm = DataLoader(test_data_norm, batch_size=64, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(pretrained=True)

# Freeze all convolutional parameters except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True  # Unfreeze the final fully connected layer

# Replace the final fully connected layer for the number of classes in the dataset
num_classes = len(train_data_norm.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model
start_time = time.time()
for epoch in range(5):  # 5 epochs for demonstration
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_norm:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data_norm)
    print(f"Epoch {epoch+1}/{5} - Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")
train_accuracy = calculate_accuracy(train_loader_norm, model)
print(train_accuracy)

Epoch 1/5 - Loss: 4.3500
Epoch 2/5 - Loss: 2.7004
Epoch 3/5 - Loss: 2.0292
Epoch 4/5 - Loss: 1.6522
Epoch 5/5 - Loss: 1.4248
Training Time: 281.53787660598755 seconds
75.97860719162495


le modèle pré-entraîné, mais en gelant uniquement les paramètres dans "layer1"avec la deuxieme de normalisation

In [ ]:






# Charger le modèle pré-entraîné ResNet18
model = resnet18(pretrained=True)

# Geler les paramètres de toutes les couches de convolution
for name, param in model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = False

# Remplacer la dernière couche entièrement connectée pour la classification par une nouvelle couche adaptée à notre nombre de classes
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Entraînement du modèle
start_time = time.time()
for epoch in range(5):  # 5 epochs pour l'exemple
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{5} - Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")
train_accuracy = calculate_accuracy(train_loader, model)
print(train_accuracy)


Epoch 1/5 - Loss: 3.3223
Epoch 2/5 - Loss: 1.7341
Epoch 3/5 - Loss: 1.1669
Epoch 4/5 - Loss: 0.8319
Epoch 5/5 - Loss: 0.5961
Training Time: 341.91364789009094 seconds
82.0323167956304


le modèle pré-entraîné, mais en gelant uniquement les paramètres dans "layer1"avec la premiere  normalisation

In [ ]:
# Charger le modèle pré-entraîné ResNet18
model = resnet18(pretrained=True)

# Geler les paramètres de toutes les couches de convolution
for name, param in model.named_parameters():
    if 'layer1' in name:
        param.requires_grad = False

# Remplacer la dernière couche entièrement connectée pour la classification par une nouvelle couche adaptée à notre nombre de classes
num_classes = len(train_data_norm.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Entraînement du modèle
start_time = time.time()
for epoch in range(5):  # 5 epochs pour l'exemple
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_norm:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data_norm)
    print(f"Epoch {epoch+1}/{5} - Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")
train_accuracy = calculate_accuracy(train_loader_norm, model)
print(train_accuracy)

le modèle pré-entraîné, mais en laissant tous les paramètres (incluant les couches de convolution) se faire ajuster par backprop) deuxieme normalisation

In [ ]:


# Charger le modèle pré-entraîné ResNet18
model = resnet18(pretrained=True)

# Remplacer la dernière couche entièrement connectée pour la classification par une nouvelle couche adaptée à notre nombre de classes
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Entraînement du modèle
start_time = time.time()
for epoch in range(5):  # 5 epochs pour l'exemple
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{5} - Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")
train_accuracy = calculate_accuracy(train_loader, model)
print(train_accuracy)


Epoch 1/5 - Loss: 3.5141
Epoch 2/5 - Loss: 1.9131
Epoch 3/5 - Loss: 1.3390
Epoch 4/5 - Loss: 0.9762
Epoch 5/5 - Loss: 0.7238
Training Time: 351.7113380432129 seconds
80.93991807009559


le modèle pré-entraîné, mais en laissant tous les paramètres (incluant les couches de convolution) se faire ajuster par backprop) premiere normalisation

In [ ]:

# Charger le modèle pré-entraîné ResNet18
model = resnet18(pretrained=True)

# Remplacer la dernière couche entièrement connectée pour la classification par une nouvelle couche adaptée à notre nombre de classes
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Entraînement du modèle
start_time = time.time()
for epoch in range(5):  # 5 epochs pour l'exemple
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader_norm:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data_norm)
    print(f"Epoch {epoch+1}/{5} - Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Training Time: {end_time - start_time} seconds")
train_accuracy = calculate_accuracy(train_loader_norm, model)
print(train_accuracy)

Epoch 1/5 - Loss: 3.5205
Epoch 2/5 - Loss: 1.9295
Epoch 3/5 - Loss: 1.3172
Epoch 4/5 - Loss: 1.0222
Epoch 5/5 - Loss: 0.7380
Training Time: 352.5062870979309 seconds
72.84934000910333
